In [ ]:
from week2_novel import *
import nbimporter, importlib
import utility_fncs, global_vars

In [ ]:

import json
from typing import get_origin, get_args, Literal

In [ ]:
# tool functions
# Define function with type hints and a specific docstring format

def get_current_weather(location: str, unit: Literal["celsius", "fahrenheit"] = "celsius"):
    """Gets the current weather in a given location.
    It can be a city, state format

    Args:
        location (str): The city and state, e.g., San Francisco, CA.
        a example if of form San Francisco, CA
        unit (str): The unit of temperature, either 'celsius' or 'fahrenheit'.
    """
    # function logic     
    return_dict={
        'type':'json',
        'content':'',
    }
    if "san francisco" in location.lower():
        return_dict['content']=json.dumps({"location": location, "temperature": "15", "unit": unit})
    elif "tokyo" in location.lower():
        return_dict['content']= json.dumps({"location": location, "temperature": "22", "unit": unit})
    else:
        return_dict['content']= json.dumps({"location": location, "temperature": "unknown"})
    return return_dict


def get_stock_price(symbol: str):
    """Retrieves the current stock price for a given ticker symbol.
    
    Args:
        symbol (str): The stock ticker symbol, e.g., AAPL for Apple, GOOG for Google.
    """
    return_dict={
        'type':'json',
        'content':'',
    }
    if symbol.upper() == "AAPL":
        return_dict['content']= json.dumps({"symbol": "AAPL", "price": "175.28"})
    else:
        return_dict['content']= json.dumps({"symbol": symbol, "price": "not found"})
    return return_dict



In [ ]:
import base64
from io import BytesIO
from PIL import Image

def create_image(msg: str):
    """"Generates an image from a detailed text prompt. Use this tool when a user explicitly asks to draw or create a picture,
    OR when a visual aid would significantly help in explaining a complex concept or answering a question."

    Args:
        msg(str) : the string that can be passed as a input to openai image generation (openai.image.generate) call's  keyword argument prompt. 
        It has to obtained from the user's message or intention requesting for a iamge
    """
    return_dict={
        'type':'image',
        'content':'',
    }
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=msg,
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return_dict['content']=Image.open(BytesIO(image_data))
    print(return_dict['content'])
    return return_dict

In [ ]:
#openai
#create_image('create image of a sunrise in the Himalayas')

In [ ]:
from openai import OpenAI
tool_fncs = []
tool_fncs.append(utility_fncs.create_tool_from_function(get_current_weather))
available_functions = {
        "get_current_weather": get_current_weather,
        "get_stock_price": get_stock_price,
        "create_image": create_image,
    }
def chat_gpt_tool_call(qt, file=None, model=None):
    #print(qt)
    #msg = utility_fncs.create_gpt_prompt(qt,"user")
    #print(msg)
    #global_vars.prompt_gpt = utility_fncs.append_conv(global_vars.prompt_gpt,msg)
    #print(global_vars.prompt_gpt)
    user_image = None
    global_vars.prompt_gpt.append({"role": "user", "content":qt})
    openai = OpenAI()
    response = openai.chat.completions.create(
        model=global_vars.model_openai_4_5_preview,
        messages=global_vars.prompt_gpt,        
        tools=tool_fncs
    )
    #print(response)
    reply = response.choices[0].message
    reply_tool_calls = reply.tool_calls
    if reply_tool_calls:
        # list to hold the reply from tool calls
        tool_messages = []
        for reply_tool_call in reply_tool_calls:
            function_name = reply_tool_call.function.name
            function_to_call = available_functions.get(function_name,None)
            if not function_to_call:
                continue
            function_args = json.loads(reply_tool_call.function.arguments)

            # call our tool
            function_response  = function_to_call(**function_args)
            if (isinstance(function_response,dict)):
                if function_response.get('type')=='image':
                    user_image = function_response.get('content')
                    function_response = "Image created as per user's request"
                else:
                    function_response=function_response.get('content','')
            tool_messages.append(
                {
                    "tool_call_id" : reply_tool_call.id,
                    "role" : "tool",
                    "name" : function_name,
                    "content" : function_response,
                }
            )
            global_vars.prompt_gpt.append(reply)
            #global_vars.prompt_gpt = utility_fncs.append_conv(global_vars.prompt_gpt, reply)
            #global_vars.prompt_gpt = utility_fncs.append_conv(global_vars.prompt_gpt, *tool_messages)
            global_vars.prompt_gpt.append(*tool_messages)
            response = openai.chat.completions.create(
                model=model_openai_4onano,
                messages=global_vars.prompt_gpt,
            )
   # print(user_image)
    return response.choices[0].message.content, user_image
    
                

        
        

In [ ]:
#global_vars.prompt_gpt = utility_fncs.reset_conv_history(global_vars.prompt_gpt,utility_fncs.create_gpt_prompt(global_vars.system_gpt_msg,'assistant'))
#chat_gpt_tool_call('weather in tokyo?')
#launcher = utility_fncs.get_gradio_launcher(chat_gpt_tool_call)
tool_fncs.append(utility_fncs.create_tool_from_function(create_image))
#global_vars.prompt_gpt = utility_fncs.reset_conv_history(global_vars.prompt_gpt,utility_fncs.create_gpt_prompt(global_vars.system_gpt_msg,'assistant'))
#launcher = utility_fncs.get_gradio_multi_modal_launcher(chat_gpt_tool_call)

In [ ]:
def chat(history,qt):
    messages = [{"role": "system", "content": global_vars.system_gpt_msg}] + history
    
    global_vars.prompt_gpt.append(history[-1])
    response = openai.chat.completions.create(model=global_vars.model_openai_4_5_preview, messages=global_vars.prompt_gpt, tools=tool_fncs)
    reply = response.choices[0].message
    reply_tool_calls = reply.tool_calls
    user_image = None
    if reply_tool_calls:
        # list to hold the reply from tool calls
        tool_messages = []
        for reply_tool_call in reply_tool_calls:
            function_name = reply_tool_call.function.name
            function_to_call = available_functions.get(function_name,None)
            if not function_to_call:
                continue
            function_args = json.loads(reply_tool_call.function.arguments)

            # call our tool
            function_response  = function_to_call(**function_args)
            if (isinstance(function_response,dict)):
                if function_response.get('type')=='image':
                    user_image = function_response.get('content')
                    function_response = "Image created as per user's request"
                else:
                    function_response=function_response.get('content','')
            tool_messages.append(
                {
                    "tool_call_id" : reply_tool_call.id,
                    "role" : "tool",
                    "name" : function_name,
                    "content" : function_response,
                }
            )
            global_vars.prompt_gpt.append(reply)
            #global_vars.prompt_gpt = utility_fncs.append_conv(global_vars.prompt_gpt, reply)
            #global_vars.prompt_gpt = utility_fncs.append_conv(global_vars.prompt_gpt, *tool_messages)
            global_vars.prompt_gpt.append(*tool_messages)
            response = openai.chat.completions.create(
                model=global_vars.model_openai_4_5_preview,
                messages=global_vars.prompt_gpt,
            )
    global_vars.prompt_gpt.append({'role':'assistant', 'content':response.choices[0].message.content})
    history = history + [{'role':'assistant', 'content':response.choices[0].message.content}]
    print(history)
    return history,user_image


In [ ]:
global_vars.prompt_gpt

In [ ]:
launcher = utility_fncs.get_new_chat_interface(chat)
launcher.launch()

In [ ]:
"""
{
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "tool_calls": [
          {
            "id": "call_abc123",
            "type": "function",
            "function": {
              "name": "get_current_weather",
              "arguments": "{\"location\": \"San Francisco, CA\"}"
            }
          },
          {
            "id": "call_xyz789",
            "type": "function",
            "function": {
              "name": "get_stock_price",
              "arguments": "{\"symbol\": \"AAPL\"}"
            }
          }
        ]
      }
    }
  ],
  ...
}
"""